In [1]:
import os, sys
sys.path.append('pytorch_sol2/')
sys.path

['/floyd/home',
 '/usr/local/lib/python37.zip',
 '/usr/local/lib/python3.7',
 '/usr/local/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/xgboost-1.0.2-py3.7.egg',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython',
 'pytorch_sol2/']

In [2]:
import torch
torch.__version__

'1.8.1+cu102'

In [3]:
from __future__ import print_function, division
import os, time, copy, sys
import pandas as pd
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from skimage import io, transform
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import pickle
#from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch.nn.functional as F

from config import *
from GalaxiesDataset import *
from rsa_loader import *
from GZ2DML import *

import pickle
from label_smoothing import LabelSmoothingCrossEntropy


In [4]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers, regularizers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

In [5]:
transf = transforms.Compose([transforms.Resize((105, 105)),
                             transforms.RandomHorizontalFlip(p=0.5),
                             transforms.RandomRotation(degrees=(0,360)),
                             transforms.RandomVerticalFlip(p=0.5),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])])

transformations = transforms.Compose([transforms.Resize((105, 105)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
train_ds = GalaxiesDataset(TRAIN_DIR, TRAIN_CSV, transform=transf)

In [7]:
#Change to 9 for 9-class test
hubble_classes = 9 

In [8]:
if hubble_classes == 9:
    gz2_9 = train_ds.classes_frame.iloc[:, 0:5]
    change = gz2_9[gz2_9['hubble_type'].isin(['E0', 'E3-5', 'E7'])].index.tolist()
    gz2_9.loc[change, 'hubble_type'] ='E'
    dumb = pd.get_dummies(gz2_9['hubble_type'])
    train_ds.classes_frame = pd.concat([gz2_9, dumb], axis=1)

In [9]:
cl=train_ds.classes_frame['hubble_type'].astype('category').cat.codes.tolist()
temp = pd.DataFrame(cl,columns=['cat'])
temp.head()

,cat
0,7
1,8
2,0
3,8
4,7


In [10]:
y = torch.from_numpy(temp.values)
train_idx, valid_idx= train_test_split(
np.arange(len(y)),
test_size=0.4,
shuffle=True,
    random_state=42,
stratify=y)

valid_idx, test_idx= train_test_split(
np.arange(len(y[valid_idx])),
test_size=0.5,
shuffle=True,
    random_state=42,
stratify=y[valid_idx])

In [11]:
size = len(train_ds)
indices = list(range(size))
split = int(np.floor(VALIDATION_SPLIT * size))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

test_split = int(np.floor(VALIDATION_SPLIT * len(train_indices)))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = train_indices[test_split:], train_indices[:test_split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler   = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=8,
                                                 sampler=train_sampler)
val_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=8,
                                                 sampler=val_sampler)
unseen_dl = DataLoader(train_ds,batch_size=BATCH_SIZE, num_workers=8,
                                                 sampler=test_sampler)
print("Total: {} Train_dl: {} Validation_dl: {} Unseen_dl: {}".format(size, len(train_dl),                                                                    
                                                              len(val_dl), len(unseen_dl)))

Total: 239573 Train_dl: 450 Validation_dl: 150 Unseen_dl: 150


/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [12]:
device = torch.device(DEVICE)

In [13]:
#Change to 'val' if only want to evaluate
train_or_val = 'val'

In [14]:
mining_function = 'pair' #'triplet' 'pair'
loss = 'pnca' #'triplet' 'contrastive' 'panc' 'pnca' 'normsoftmax'

In [15]:
if train_or_val == 'train':
    model = torch.load('./models/gz2_label_smoothing_128')    
    del model.fc[6]
    del model.fc[5]
    del model.fc[4]
if train_or_val == 'val':
    pth = './models/gz2{}_{}_128'.format(mining_function, loss)
    model = torch.load(pth)

In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.05, nesterov=True, weight_decay=0.0005)
criterion = LabelSmoothingCrossEntropy(reduction='mean')
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3,threshold=0.0001,factor=0.1, verbose=True)

In [17]:
gz2DML = GZ2DML(train_dl, val_dl, unseen_dl, model, optimizer, scheduler, criterion,mining_function, loss,
                  hubble_classes=hubble_classes, device=device,BATCH_SIZE=BATCH_SIZE)
if train_or_val == 'train':
    gz2DML.train(n_epochs=300)

In [18]:
if hubble_classes == 9:
    test_ds = GalaxiesDataset_rsa(TEST_DIR, TEST_CSV, transform=transformations)
elif hubble_classes == 11:
    test_ds = GalaxiesDataset_rsa(TEST_DIR, './classes/usethis_rsa2.csv', transform=transformations)

In [19]:
size = len(test_ds)
indices = list(range(size))
test_sampler = SubsetRandomSampler(indices)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=4,
                                                     sampler=test_sampler)
print("Total: {} Test_dl: {}".format(size, len(test_dl)))    

Total: 1249 Test_dl: 4


In [ ]:
res = gz2DML.RSAtest(test_dl)

Evaluation on RSA test set beginning:
